In [46]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import haversine as hs
import requests
import reverse_geocoder
from geopy.geocoders import Nominatim
from tqdm import tqdm

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

In [1]:
import pandas as pd

coordinates_df = pd.read_csv("/Users/jangaydoul/Desktop/Copenhagen Business School/4. Semester :: Thesis/03_Data/coordinates_df.csv")

In [2]:
len(coordinates_df)

162113

In [74]:
coordinates_df_7_65000_74999 = coordinates_df.iloc[65000:75000] # DONE
coordinates_df_8_75000_84999 = coordinates_df.iloc[75000:85000] # DONE
coordinates_df_9_85000_94999 = coordinates_df.iloc[85000:95000] # DONE
coordinates_df_10_95000_104999 = coordinates_df.iloc[95000:105000]
coordinates_df_11_105000_114999 = coordinates_df.iloc[105000:115000]
coordinates_df_12_115000_124999 = coordinates_df.iloc[1155000:125000]
coordinates_df_13_125000_134999 = coordinates_df.iloc[125000:135000]
coordinates_df_14_135000_144999 = coordinates_df.iloc[135000:145000]
coordinates_df_15_145000_154999 = coordinates_df.iloc[145000:155000]
coordinates_df_16_155000_1162113 = coordinates_df.iloc[155000:]

In [82]:
# 3) Calculate distances using Bing Matrix API // Batches

# Define Bing Maps API key
api_key = 'AjjPXgFK_kEPapm4Aa71lVBC-OD0Cv--TxfWszdsMyVPAptZVtzAxV8a52UjW6Po'

# Define the Bing Maps API endpoint
url = 'https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix'

# set the timeout value to 60 seconds to prevent errors from short-term connection issues
# (default is 10 seconds)
timeout = 60

# Define a function to calculate distance between two locations using Bing Maps API
def calculate_distances(rows):
    distances = []
    with requests.Session() as session:
        for index, row in rows.iterrows():
            # Define the request parameters
            params = {
                'origins': f"{row['FromLatitude']},{row['FromLongitude']}",
                'destinations': f"{row['ToLatitude']},{row['ToLongitude']}",
                'travelMode': 'driving',
                'key': api_key
            }
            # Send the API request and get the response
            response = session.get(url, params=params, timeout=timeout).json()
            # Extract the distance from the response
            distance = response['resourceSets'][0]['resources'][0]['results'][0]['travelDistance']
            distances.append(distance)
    return distances

# Define the batch size
batch_size = 100

# Split the DataFrame into batches and calculate distances for each batch
batches = [coordinates_df_10_95000_104999[i:i+batch_size] for i in range(0, len(coordinates_df_10_95000_104999), batch_size)]
distances = []
for batch in batches:
    distances += calculate_distances(batch)

# Add the distances to the DataFrame
coordinates_df_10_95000_104999['route_distance'] = distances

# Print the updated DataFrame
print(coordinates_df_10_95000_104999.head())

# Save the DataFrame as a CSV file
coordinates_df_10_95000_104999.to_csv('coordinates_df_10_95000_104999.csv', index=False)

ReadTimeout: HTTPSConnectionPool(host='dev.virtualearth.net', port=443): Read timed out. (read timeout=60)

In [77]:
coordinates_df_10_95000_104999

,FromLatitude,FromLongitude,ToLatitude,ToLongitude
95000,56.07587,12.75754,56.05531,14.59051
95001,60.17455,-1.15693,53.18885,-2.43895
95002,53.77629,-1.52585,53.53919,-1.03753
95003,54.08248,9.66632,53.86023,8.71934
95004,51.59228,5.01785,52.49522,-1.88211
...,...,...,...,...
104995,49.21549,7.03333,60.31150,5.38944
104996,54.63374,-6.14595,50.85454,3.18812
104997,53.25532,-6.21368,57.64718,11.95559
104998,40.94570,14.39861,50.01823,21.96463
